In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd drive/MyDrive/Colab-workspace/

/content/drive/MyDrive/Colab-workspace


In [2]:
# Setup up YOLOv5 Environment
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U -r yolov5/requirements.txt  # install dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 10001, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 10001 (delta 0), reused 1 (delta 0), pack-reused 10000
Receiving objects: 100% (10001/10001), 10.30 MiB | 9.72 MiB/s, done.
Resolving deltas: 100% (6939/6939), done.
     |████████████████████████████████| 11.2 MB 7.8 MB/s 
     |████████████████████████████████| 15.7 MB 44.8 MB/s 
     |████████████████████████████████| 60.3 MB 63 kB/s 
     |████████████████████████████████| 3.1 MB 41.8 MB/s 
     |████████████████████████████████| 596 kB 55.8 MB/s 
     |████████████████████████████████| 62 kB 798 kB/s 
     |████████████████████████████████| 38.2 MB 44 kB/s 
     |████████████████████████████████| 11.3 MB 53.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uni

In [3]:
%cd yolov5

/content/yolov5


In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

Tesla P100-PCIE-16GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [6]:
import json

from tqdm import tqdm
from utils.general import download, Path

dir = Path('../datasets/Argoverse')  # dataset root dir
urls = ['https://argoverse-hd.s3.us-east-2.amazonaws.com/Argoverse-HD-Full.zip']
download(urls, dir=dir, delete=False)


def argoverse2yolo(set):
    labels = {}
    a = json.load(open(set, "rb"))
    for annot in tqdm(a['annotations'], desc=f"Converting {set} to YOLOv5 format..."):
        img_id = annot['image_id']
        img_name = a['images'][img_id]['name']
        img_label_name = img_name[:-3] + "txt"

        cls = annot['category_id']  # instance class id
        x_center, y_center, width, height = annot['bbox']
        x_center = (x_center + width / 2) / 1920.0  # offset and scale
        y_center = (y_center + height / 2) / 1200.0  # offset and scale
        width /= 1920.0  # scale
        height /= 1200.0  # scale

        img_dir = set.parents[2] / 'Argoverse-1.1' / 'labels' / a['seq_dirs'][a['images'][annot['image_id']]['sid']]
        if not img_dir.exists():
            img_dir.mkdir(parents=True, exist_ok=True)

        k = str(img_dir / img_label_name)
        if k not in labels:
            labels[k] = []
        labels[k].append(f"{cls} {x_center} {y_center} {width} {height}\n")

    for k in labels:
        with open(k, "w") as f:
            f.writelines(labels[k])

  0%|          | 0.00/29.2G [00:00<?, ?B/s]

Unzipping ../datasets/Argoverse/Argoverse-HD-Full.zip...


In [7]:
annotations_dir = 'Argoverse-HD/annotations/'
(dir / 'Argoverse-1.1' / 'tracking').rename(dir / 'Argoverse-1.1' / 'images')  # rename 'tracking' to 'images'
for d in "train.json", "val.json":
    argoverse2yolo(dir / annotations_dir / d)

Converting ../datasets/Argoverse/Argoverse-HD/annotations/train.json to YOLOv5 format...: 100%|██████████| 771774/771774 [00:33<00:00, 22951.19it/s]
Converting ../datasets/Argoverse/Argoverse-HD/annotations/val.json to YOLOv5 format...: 100%|██████████| 253941/253941 [00:11<00:00, 22622.29it/s]


In [ ]:
!python train.py --weights /content/yolov5/runs/train/exp/weights/best.pt --data Argoverse.yaml --epochs 20 --batch-size 64 --resume

train: weights=/content/yolov5/runs/train/exp/weights/best.pt, cfg=, data=Argoverse.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=20, batch_size=64, imgsz=640, rect=False, resume=True, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
Resuming training from ./runs/train/exp2/weights/last.pt
YOLOv5 🚀 v6.0-100-g5185981 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, 

##Evaluate Model

In [10]:
!python val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --data Argoverse.yaml

val: data=/content/yolov5/data/Argoverse.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-102-gf17c86b torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 213 layers, 7031701 parameters, 0 gradients, 15.9 GFLOPs
val: Scanning '../datasets/Argoverse/Argoverse-1.1/labels/val/00c561b9-2057-358d-82c6-5b06d76cebcf/ring_front_center' images and labels...15055 found, 7 missing, 0 empty, 0 corrupted: 100% 15062/15062 [00:24<00:00, 613.79it/s]
val: WARNING: ../datasets/Argoverse/Argoverse-1.1/images/val/1d676737-4110-3f7e-bec0-0c90f74c248f/ring_front_center/ring_front_center_315984819687787736.jpg: 1 duplicate labels removed
val: New cache created: ../datasets/Argover

## Save Model to Drive

In [ ]:
!cp -r /content/yolov5/runs /content/drive/MyDrive/Colab-workspace/runs